In [1]:
import csv
import os
from itertools import batched

from dotenv import load_dotenv
from tqdm.notebook import tqdm

from bigger_picker.rayyan import RayyanManager

load_dotenv()

True

In [2]:
def get_csv_ratings(file_path):
    dois = set()
    with open(file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            dois.add(row["doi"])

    dois = {doi.removeprefix("https://doi.org/").lower().strip() for doi in dois}
    return dois

In [3]:
included_ft_dois = get_csv_ratings("Sanders - included.csv")
excluded_ft_dois = get_csv_ratings("Sanders - excluded-ft.csv")
excluded_ab_dois = get_csv_ratings("Sanders - excluded-ab.csv")
failed_ft_dois = get_csv_ratings("Sanders - failed-ft.csv")
missing_ft_dois = get_csv_ratings("Sanders - missing-ft.csv")

# Combine the ones that indicate inclusion at the abstract level
included_ab_dois = (
    included_ft_dois.union(excluded_ft_dois)
    .union(failed_ft_dois)
    .union(missing_ft_dois)
)
print(f"Included at abstract level: {len(included_ab_dois)}")
print(f"Included at full text level: {len(included_ft_dois)}")

Included at abstract level: 5520
Included at full text level: 1983


In [4]:
rayyan = RayyanManager("../rayyan_tokens.json")

In [5]:
n_articles = 29610
batch_size = 1000
batches = batched(range(0, n_articles), batch_size)
total_batches = (n_articles + batch_size - 1) // batch_size

with tqdm(total=total_batches, desc="Overall Progress") as overall_pbar:
    for batch_idx, batch in enumerate(batches):
        results_params = {"start": batch[0], "length": len(batch)}
        articles = rayyan.review.results(rayyan.review_id, results_params)
        articles_data = articles["data"]

        with tqdm(
            total=len(articles_data),
            desc=f"Batch {batch_idx + 1}/{total_batches}",
            leave=False,
        ) as batch_pbar:
            for article in articles_data:
                article_doi = article["doi"]
                try:
                    article_doi = (
                        article_doi.removeprefix("https://doi.org/").lower().strip()
                    )
                except AttributeError:
                    # No DOI so skip
                    batch_pbar.update(1)
                    continue

                plan = {}

                if article_doi in included_ft_dois:
                    plan["OttoSR: Include Full-Text"] = 1
                    plan["OttoSR: Include Abstract"] = 1
                if article_doi in included_ab_dois:
                    plan["OttoSR: Include Abstract"] = 1
                if article_doi in excluded_ft_dois:
                    plan["OttoSR: Exclude Full-Text"] = 1

                # Check if we can avoid processing
                existing_labels = article["customizations"].get("labels", {})

                if (
                    existing_labels.get("OttoSR: Include") is None
                    and existing_labels.get("OttoSR: Exclude Full-Text") is None
                    and plan.get("OttoSR: Include Abstract") is None
                    and plan.get("OttoSR: Exclude Full-Text") is None
                ):
                    # There is no existing label or plan for this article
                    batch_pbar.update(1)
                    continue

                # If we haven't set the plan, we default to removing existing labels
                plan.setdefault("OttoSR: Include", -1)
                plan.setdefault("OttoSR: Exclude Full-Text", -1)

                rayyan.review.customize(rayyan.review_id, article["id"], plan)
                included_ab_dois.discard(article_doi)
                included_ft_dois.discard(article_doi)
                excluded_ft_dois.discard(article_doi)

                batch_pbar.update(1)

        overall_pbar.update(1)

Overall Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Batch 1/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 2/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 3/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 4/30:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Batch 26/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 27/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 28/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 29/30:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 30/30:   0%|          | 0/610 [00:00<?, ?it/s]

In [7]:
print("Included abstract DOIs left:", len(included_ab_dois))
print("Included full-text DOIs left:", len(included_ft_dois))
print("Excluded DOIs left:", len(excluded_ft_dois))


Included abstract DOIs left: 77
Included full-text DOIs left: 19
Excluded DOIs left: 18


In [59]:
def get_leftover_dois(csv_path, leftover_dois):
    leftover_rows = []
    with open(csv_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if (
                row["doi"].removeprefix("https://doi.org/").lower().strip()
                in leftover_dois
            ):
                leftover_rows.append(row)

    with open(os.path.splitext(csv_path)[0] + "-leftover.csv", "w") as csvfile:
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in leftover_rows:
            writer.writerow(row)

    return leftover_rows


leftover_included = get_leftover_dois("Sanders-included.csv", included_dois)
assert len(leftover_included) == len(included_dois)
leftover_excluded = get_leftover_dois("Sanders-excluded_ft.csv", excluded_dois)
assert len(leftover_excluded) == len(excluded_dois)

In [60]:
for article in leftover_included:
    print(
        f"""
Article DOI: {article["doi"]}
Title: {article["title"]}
Authors: {article["authors"]}
Year: {article["date"]}
"""
    )


Article DOI: https://doi.org/10.1109/cisti.2014.6877016
Title: Electronic games for curricular learning in mother and foreign languages
Authors: ["Moura, A."]
Year: 2014


Article DOI: https://doi.org/10.21071/pse.v15i3.16093
Title: Positive Internet use and online civic engagement versus active involvement in selected online risks – how are both connected in adolescents from six European countries?
Authors: ["Pyżalski, J."]
Year: 2023


Article DOI: https://doi.org/10.19173/irrodl.v20i2.4074
Title: Exploring high school students' educational use of youtube
Authors: ["Bardakci, S."]
Year: 2019


Article DOI: https://doi.org/10.1007/s10803-022-05694-w
Title: Correction: Video Game Use, Aggression, and Social Impairment in Adolescents with Autism Spectrum Disorder (Journal of Autism and Developmental Disorders, (2022), 10.1007/s10803-022-05649-1)
Authors: ["Davis, K.","Iosif, A.-M.","Nordahl, C.W.","Solomon, M.","Krug, M.K."]
Year: 2023


Article DOI: https://doi.org/10.1038/s41598-025-

In [61]:
for article in leftover_excluded:
    print(
        f"""
Article DOI: {article["doi"]}
Title: {article["title"]}
Authors: {article["authors"]}
Year: {article["date"]}
"""
    )


Article DOI: https://doi.org/10.18687/laccei2018.1.1.362
Title: Game based learning in improving students' derivative calculation skills
Authors: ["Zhu, W.","Zhang, L.","Brown, W.","Dabipi, I.","Peterson, E.","Joshi, R.","Bayingana, A."]
Year: 2018


Article DOI: https://doi.org/10.34190/ecsm.10.1.1151
Title: The Influence of TikTok on Young Generation in Vietnam
Authors: ["Truong, P.H.","Kim, A.D."]
Year: 2023


Article DOI: https://doi.org/10.32870/cys.v0i33.7007
Title: Informal learning strategies for transmedia skills in adolescents in Uruguay
Authors: ["Morales, S.","Cabrera, M.","Rodríguez, G."]
Year: 2018


Article DOI: https://doi.org/10.1038/s44184-022-00011-w
Title: The mental health and well-being profile of young adults using social media
Authors: ["Di Cara, N.H.","Winstone, L.","Sloan, L.","Davis, O.S.P.","Haworth, C.M.A."]
Year: 2022


Article DOI: https://doi.org/10.1109/mmul.2015.96
Title: Social Media Interaction and Analytics for Enhanced Educational Experiences
Auth